# Defining and Calling Python Functions
### Miki Tebeka : CEO, CTO & UFO @ 353Solutions

We're going to cover the many ways you can define and call functions in Python. I hope each of you will learn something new.

In [1]:
%load_ext watermark
%watermark

11/02/2015 22:00:05

CPython 3.5.0
IPython 4.0.0

compiler   : GCC 5.2.0
system     : Linux
release    : 4.2.5-1-ARCH
machine    : x86_64
processor  : 
CPU cores  : 4
interpreter: 64bit


In [2]:
def sub(x, y):
    """Return the y's predecessor of x"""
    return x - y

In [3]:
print(sub(1, 7))  # "regular" call
print(sub(y=7, x=1)) # call with keyword arguments
print(sub(1, y=7))  # mix & match

-6
-6
-6


In [4]:
# however you can't
print(sub(y=7, 1))

SyntaxError: positional argument follows keyword argument (<ipython-input-4-0b8706576d85>, line 2)

In [5]:
args = (1, 7)
print(sub(*args))

# * in function call unpacks sequence to positional

-6


In [6]:
def vargs(*args):
    print(args)
    
vargs()
vargs(1,2,3)

a, b = (1, 2), (3, 4)
vargs(*a, *b)  # New in 3.5 (used to be: vargs(*(a+b)))

# * in function definition packs positional to tuple

()
(1, 2, 3)
(1, 2, 3, 4)


In [7]:
kw = {'x': 1, 'y': 7}
print(sub(**kw))

# ** in function call unpacks mapping to keyword args

-6


In [8]:
def kwargs(**kw):
    print(kw)
    
kwargs()
kwargs(x=1, y=2)

kw1, kw2 = {'x': 1, 'y': 2}, {'a': 10, 'b': 20}
kwargs(**kw1, **kw2)  # New in 3.5 (can't have duplicate keys)

# ** in function definition packs keyword args to dict

{}
{'y': 2, 'x': 1}
{'y': 2, 'a': 10, 'x': 1, 'b': 20}


In [9]:
def eat_all(*args, **kw):
    print(args)
    print(kw)

eat_all()
eat_all('bugs')
eat_all(taz='brown')
eat_all('daffy', 'taz', colors=['black', 'brown'])

# very bad design, mostly used in decorators

()
{}
('bugs',)
{}
()
{'taz': 'brown'}
('daffy', 'taz')
{'colors': ['black', 'brown']}


In [10]:
def sub(x, y=7):
    return x - y

print(sub(1, 7))
print(sub(1))

# use x=y to speficy default arguments

-6
-6


In [24]:
def prappend(n, vals=[]):
    vals.append(n)
    print(vals)
prappend(1)
prappend(2)

# default arguments are computed at definition time
# NEVER USE MUTABLE DEFAULT ARGUMENTS

[1]
[1, 2]


In [25]:
# fix to the above
def prappend(n, vals=None):
    vals = [] if vals is None else vals
    vals.append(n)
    print(vals)
prappend(1)
prappend(2)

[1]
[2]


In [11]:
class subber:
    def __init__(self, n):
        self.n = n

    def __call__(self, x):
        return self.n + x

sub10 = subber(10)
sub10(7)

# any object which implements __call__ can be used as a function
# (yes, functions objects have __call__ as well)

17

In [12]:
def make_subber(n):
    def subber(x):
        return x - n
    return subber

sub7 = make_subber(7)
sub7(1)

# closure

-6

In [13]:
ncalls = 0
def sub(x, y):
    ncalls += 1
    return x + y

sub(1, 2)  # UnboundLocalError, can't reassign to global

UnboundLocalError: local variable 'ncalls' referenced before assignment

In [14]:
ncalls = 0
def sub(x, y):
    global ncalls
    ncalls += 1
    return x - y
print(sub(1, 7))
print(ncalls)

# Use the "global" keyword for such things

-6
1


In [15]:
from time import time
call_times = []
def sub(x, y):
    call_times.append(time())
    return x - y
sub(1, 7)
print(call_times)

# However you *can* mutate an object without using "global"

[1446494434.27457]


In [16]:
def sub(x, y, *, verbose=False):
    if verbose:
        print('%s - %s calculated' % (x,  y))
    return x - y

print(sub(1, 7))
print(sub(1, 7, verbose=True))

# * forces keyword argument only (see below), new in 3.5

-6
1 - 7 calculated
-6


In [17]:
sub(1, 7, True)

TypeError: sub() takes 2 positional arguments but 3 were given

In [18]:
def sub(x: int, y: int) -> int:
    return x - y

print(sub(1, 7))
sub.__annotations__

# type annotation, just store information in __annotations__
# new in 3.5

-6


{'return': int, 'x': int, 'y': int}

In [19]:
async def sub(x, y):
    return x - y

print(sub(1, 7))

# new in 3.5, see PEP 492

<coroutine object sub at 0x7f32b86fffc0>


/usr/lib/python3.5/site-packages/ipykernel/__main__.py:4: RuntimeWarning: coroutine 'sub' was never awaited


In [20]:
def sub(x, y):
    """Returns the y's predecessor to x

    >>> sub(10, 3)
    7
    >>> sub(5, 22)
    -17
    """
    return x - y

import doctest
doctest.testmod()

# You can write test in your docstring
# nose, py.test and Sphinx can test your documentation

TestResults(failed=0, attempted=2)

In [21]:
import re

is_comment = re.compile('^\s*#').match
print(is_comment(' # this is a comment'))

# You can use bound methods as functions

<_sre.SRE_Match object; span=(0, 2), match=' #'>


In [22]:
from functools import partial

compact_ws = partial(re.compile('\s+').sub, ' ')
compact_ws('how   are  \t you?')

# You can use functool.partial to create new functions
# (AKA currying)

'how are you?'

In [23]:
from functools import wraps
from time import time, sleep

def timed(fn):
    @wraps(fn)
    def wrapper(*args, **kw):
        start = time()
        try:
            return fn(*args, **kw)
        finally:
            print('%s took %.2fsec' % (fn.__name__, time() - start))
    return wrapper
            
@timed
def sub(x, y):
    sleep(0.1)
    return x - y

print(sub(1, 7))

# you can use decorators to add functionality to functions

sub took 0.10sec
-6
